# llama_3.1 8B Instruct

This model could ben downloaded and deploy locally, despite its slow runtime. 

In [ ]:
import json
import torch
import transformers
from huggingface_hub import snapshot_download, login

In [19]:
with open("api.json") as f:
    login(token=json.load(f)["HF_TOKEN"])

In [ ]:
local_dir = snapshot_download(repo_id="meta-llama/Llama-3.2-3B-Instruct")
local_dir

Fetching 16 files: 100%|██████████| 16/16 [00:00<00:00, 208412.62it/s]


'/Users/ivan/.cache/huggingface/hub/models--meta-llama--Llama-3.2-3B-Instruct/snapshots/0cb88a4f764b7a12671c53f0838cd831a0843b95'

In [21]:

pipeline: transformers.pipeline = transformers.pipeline("text-generation", model=local_dir, device_map="auto", model_kwargs={"torch_dtype": torch.float16, "low_cpu_mem_usage": True})
terminators = [pipeline.tokenizer.eos_token_id, pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")]

Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.49s/it]
Some parameters are on the meta device because they were offloaded to the disk.
Device set to use mps


In [22]:
# takes a str query and returns str responses
def get_response(query, message_history=[], max_tokens=2048, temperature=0.6, top_p=0.9):
    user_prompt = message_history + [{"role": "user", "content": query}]
    prompt = pipeline.tokenizer.apply_chat_template(
        user_prompt, tokenize=False, add_generation_prompt=True
    )
    outputs = pipeline(
        prompt,
        max_new_tokens=max_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=temperature,
        top_p=top_p
    )
    response = outputs[0]["generated_text"][len(prompt):]
    return response

# take a list of messages (dict[str, any] -> role, content) and a response
def response_from_msg_list(msg_list: list[dict], message_history=[], max_tokens=2048, temperature=0.6, top_p=0.9):
    flat_msg_list = []
    for x in range(len(msg_list)): flat_msg_list += [{"role":k , "content": v} for k, v in msg_list[x].items()]
    user_prompt = message_history + flat_msg_list
    print("message list:", user_prompt)
    prompt = pipeline.tokenizer.apply_chat_template(
        user_prompt, tokenize=False, add_generation_prompt=True
    )
    outputs = pipeline(
        prompt,
        max_new_tokens=max_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=temperature,
        top_p=top_p
    )
    response = outputs[0]["generated_text"][len(prompt):]
    return response

In [26]:
get_response("What is the capital of Canada in one word and what's your confidence as a percentage in parentheses that this is the correct answer?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Ottawa (100%)'

In [36]:
response_from_msg_list([{"user": "What is the richest city in Australia?"}, {"system": "as concisely as possible and your confidence as a percentage in parentheses at the end"}])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


message list: [{'role': 'user', 'content': 'What is the richest city in Australia?'}, {'role': 'system', 'content': 'as concisely as possible and your confidence as a percentage in parentheses at the end'}]


'Melbourne (92%)'